In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [4]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [5]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [6]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"title": "Description", "description": "\uc8fc\uc81c\uc5d0 \ub300\ud55c \uac04\uacb0\ud55c \uc124\uba85", "type": "string"}, "hashtags": {"title": "Hashtags", "description": "\ud574\uc2dc\ud0dc\uadf8 \ud615\uc2dd\uc758 \ud0a4\uc6cc\ub4dc(2\uac1c \uc774\uc0c1)", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [7]:
# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

In [8]:
# 타입을 확인합니다.
type(answer)

dict

In [9]:
# answer 객체를 출력합니다.
answer

{'description': '지구 온난화는 지구의 평균 기온이 지속적으로 상승하는 현상으로, 이는 주로 인간 활동에 의해 발생하는 온실가스 배출로 인해 발생합니다. 이로 인해 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하고 있습니다. 이러한 변화는 생태계와 인간 사회에 심각한 영향을 미치고 있습니다.',
 'hashtags': '#지구온난화 #기후변화 #온실가스 #환경보호'}

- Pydantic 을 사용하지 않고 JsonOutputParser 를 사용

Pydantic 없이도 이 기능을 사용할 수 있습니다. 이 경우 JSON을 반환하도록 요청하지만, 스키마가 어떻게 되어야 하는지에 대한 구체적인 정보는 제공하지 않습니다.

In [10]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)

{'description': '지구 온난화는 대기 중 온실가스 농도가 증가하면서 지구의 평균 기온이 상승하는 현상을 말합니다. 이는 기후 변화, 해수면 상승, 극지방 빙하의 감소 등 다양한 환경 문제를 초래합니다.', 'hashtags': ['#지구온난화', '#기후변화', '#온실가스', '#환경문제', '#해수면상승']}
